## Functions for Converting Annotations

In this notebook, we will define two functions to handle the conversion of binary mask annotations to JSON files. The first function will convert binary mask annotations to a JSON file that is not compatible with QuPath. The second function will convert this non-compatible JSON file to a QuPath-compatible JSON file.

### 1. Convert Binary Mask Annotation to Non-QuPath-Compatible JSON

This function will take a binary mask annotation as input and generate a JSON file that is not compatible with QuPath. The steps involved in this function include:

1. Reading the binary mask annotation.
2. Extracting contours from the binary mask.
3. Creating a JSON structure to store the contours.
4. Saving the JSON file.

### 2. Convert Non-QuPath-Compatible JSON to QuPath-Compatible JSON

This function will take the non-compatible JSON file generated by the first function and convert it to a QuPath-compatible JSON file. The steps involved in this function include:

1. Reading the non-compatible JSON file.
2. Modifying the JSON structure to match QuPath's requirements.
3. Saving the QuPath-compatible JSON file.

In [2]:
import json
import glob
import os
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon
from tqdm import tqdm
import csv
import cv2
import argparse
from skimage import color
import openslide
import re
import shutil


In [3]:
def get_coor(filename, x_roi, y_roi):
    # Read the binary mask image in grayscale mode
    binary_mask = cv2.threshold(cv2.imread(filename, cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)[1]
    
    # Find contours in the binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # Process each contour to adjust coordinates based on ROI offsets
    final_contours = [
        [[(c[0]) + x_roi, (c[1]) + y_roi] for c in np.squeeze(contour).tolist()]
        for contour in contours if len(contour) > 5  # Filter out small contours
    ]
    
    return final_contours


# Convert Binary mask annotation to Non-QuPath-Compatible JSON

In [4]:
input_folder = '/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/tubule_primitive/2'
output_folder = '/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/'
folders = glob.glob(f'{input_folder}*')

for folder in folders:
    print(folder)
    geojson_features = []
    bioid = os.path.basename(folder)
    print(bioid)
    
    json_path = f'{output_folder}{bioid}.json'
    
    # Check if the JSON file already exists
    if not os.path.exists(json_path):
        data = {}
        
        # Get all tubule image files in the folder
        tubule_files = glob.glob(f'{folder}/single_tubule/tubule/*.png')
        
        # Process each tubule file
        for tubule_file in tqdm(tubule_files, desc=f'Processing TUBULE {os.path.basename(folder)}'):
            bbox_image = os.path.basename(tubule_file)
            data[bbox_image] = {}

            # Extract ROI coordinates from the filename
            x_roi = int(bbox_image.split('x=')[1].split(',')[0])
            y_roi = int(bbox_image.split('y=')[1].split(',')[0])

            # Define paths for related structure files
            lumen_file = tubule_file.replace('/tubule/', '/lumen/')
            nuclei_file = tubule_file.replace('/tubule/', '/cell_seg/')
            te_lumen_file = tubule_file.replace('/tubule/', '/TE+lumen/')
            structures = [tubule_file, lumen_file, nuclei_file, te_lumen_file]
            cates = ['tubule', 'lumen', 'nuclei', 'te+lumen']

            # Process each structure file
            for structure_name, cate in zip(structures, cates):
                data[bbox_image][cate] = []
                
                # Get contours for the structure and adjust coordinates
                final_contours = get_coor(structure_name, x_roi, y_roi)
                
                # Append contours to the data dictionary
                data[bbox_image][cate].append({"contours": final_contours})
        
        # Save the data dictionary as a JSON file
        with open(json_path, 'w') as json_file:
            json.dump(data, json_file, indent=4)


In [2]:

files = glob.glob('/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/*')
cates = ['tubule','lumen','nuclei','te+lumen']
for file in files:
    print(file)
    basename = os.path.basename(file)
    geojson_features = []
    with open(file, 'r') as file:
        data_raw = json.load(file)

    # Now `data` is a Python dictionary containing your contours
#     print(data_raw)
    for data in data_raw:
        tubule_info = data_raw[data]
        for cate in cates:
            primitive_contours = tubule_info[cate][0]['contours']
            for contour in primitive_contours:
                contour_coords = np.squeeze(contour).tolist()
                # contour_coords = [[(c[0]), (c[1]) + y_roi] for c in contour_coords]
                contour_coords.append(contour_coords[0])
                # Create GeoJSON feature for image contour
                geojson_feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [contour_coords]
                    },
                    "properties": {
                        "objectType": "annotation",
                        "classification": {
                            "name": f"{cate}",
                            "color": [12, 12, 96]
                        }
                    }
                }
                geojson_features.append(geojson_feature)
    with open(f'/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/qupath/{basename}', 'w') as f:
        json.dump(geojson_features, f)

/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4807_A_0038331.json
/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4817_A_0038157.json
/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4821_A_0038149.json
/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4888_A_0038033.json
/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4899_A_0043635.json
/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4919_A_0038585.json
/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4943_A_0044755.json
/Volumes/EXTERNAL_USB 1/AI_tubule/CureGN/USE_TUBULE/2_4946_A_0039486.json


KeyboardInterrupt: 